<a href="https://colab.research.google.com/github/alessioborgi/RL_Project/blob/main/X_GNN_MUTAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# X-GNN: Model-Explanations of GNNs using RL

### *Alessio Borgi*
### *Francesco Danese*

### 0: INSTALLING & IMPORTING LIBRARIES

In [25]:
!pip install torch_geometric networkx matplotlib


In [26]:
import os
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import networkx as nx
from torch.nn import Linear
import plotly.express as px
import matplotlib.pyplot as plt
import torch.nn.functional as F
import plotly.graph_objects as go
from torch_geometric.nn import MessagePassing
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx, to_dense_adj
from torch_geometric.utils import add_self_loops, remove_self_loops, degree

In [27]:
# Ensures that dataset splitting, model initialization, and training are deterministic.
def set_seed(seed):
    torch.manual_seed(seed)  # Fix seed for PyTorch (CPU).
    torch.cuda.manual_seed(seed)  # Fix seed for PyTorch (GPU).
    torch.cuda.manual_seed_all(seed)  # Fix seed for all GPUs.
    np.random.seed(seed)  # Fix seed for NumPy.
    random.seed(seed)  # Fix seed for Python's random module.
    torch.backends.cudnn.deterministic = True  # Ensure deterministic GPU behavior.
    torch.backends.cudnn.benchmark = False  # Disable cuDNN auto-tuning to enforce determinism.

set_seed(42)  # Set seed to ensure reproducibility across runs.

### 1: DATASET EXPLORATION

In [28]:
# Load the MUTAG Dataset.
dataset = TUDataset(root='data/TUDataset', name='MUTAG')

In [29]:
# Initialize lists to collect dataset-wide statistics.
num_nodes = []
num_edges = []
labels = []

# Gather data about all graphs.
for data in dataset:
    num_nodes.append(data.num_nodes)
    num_edges.append(data.num_edges)
    labels.append(data.y.item())

# Create a summary DataFrame.
df = pd.DataFrame({
    "Graph ID": range(len(dataset)),
    "Num Nodes": num_nodes,
    "Num Edges": num_edges,
    "Label": labels
})

# Dataset Statistics.
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f"Number of Graphs: {len(dataset)}")
print(f"Number of Classes: {dataset.num_classes}")
print(f"Average Nodes per Graph: {sum(num_nodes)/len(num_nodes):.2f}")
print(f"Average Edges per Graph: {sum(num_edges)/len(num_edges):.2f}")
print("Class Distribution:")
print(df['Label'].value_counts())


Dataset: MUTAG(188):
Number of Graphs: 188
Number of Classes: 2
Average Nodes per Graph: 17.93
Average Edges per Graph: 39.59
Class Distribution:
Label
1    125
0     63
Name: count, dtype: int64


In [30]:
# Plot class distribution using Plotly.
class_counts = df['Label'].value_counts().reset_index()
class_counts.columns = ['Label', 'Count']

fig = px.bar(
    class_counts,
    x='Label', y='Count',
    labels={"Label": "Class Label", "Count": "Count"},
    title="Class Distribution in MUTAG Dataset"
)
fig.show()


In [31]:
# Take the first graph of the Dataset.
single_graph = dataset[0]
print()
print(single_graph)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {single_graph.num_nodes}')
print(f'Number of edges: {single_graph.num_edges}')
print(f'Average node degree: {single_graph.num_edges / single_graph.num_nodes:.2f}')
print(f'Has isolated nodes: {single_graph.has_isolated_nodes()}')
print(f'Has self-loops: {single_graph.has_self_loops()}')
print(f'Is undirected: {single_graph.is_undirected()}')


Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [32]:
import torch
import plotly.graph_objs as go
import networkx as nx
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import numpy as np

# Assuming you already have the graph data loaded as `single_graph`
# Convert the first graph in the dataset to NetworkX.
G = to_networkx(single_graph, to_undirected=True)

# Extract the node feature matrix
node_features = single_graph.x  # Shape: [num_nodes, num_features]

# Convert one-hot encoded node features to atom types (indices)
node_colors = torch.argmax(node_features, dim=1).numpy()

# Define a colormap for different atom types
cmap = plt.get_cmap("tab10")  # You can choose any matplotlib colormap
unique_atom_types = np.unique(node_colors)
colors = {atom_type: cmap(i / len(unique_atom_types)) for i, atom_type in enumerate(unique_atom_types)}

# Generate 2D layout
pos_2d = nx.spring_layout(G, seed=42)

# Prepare edge trace for 2D visualization
edge_x_2d = []
edge_y_2d = []
for edge in G.edges():
    x0, y0 = pos_2d[edge[0]]
    x1, y1 = pos_2d[edge[1]]
    edge_x_2d.extend([x0, x1, None])
    edge_y_2d.extend([y0, y1, None])

edge_trace_2d = go.Scatter(
    x=edge_x_2d, y=edge_y_2d,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Prepare node trace for 2D visualization
node_x_2d = []
node_y_2d = []
node_labels = []
node_colors_plotly = []

for node in G.nodes():
    x, y = pos_2d[node]
    node_x_2d.append(x)
    node_y_2d.append(y)
    atom_type_list = ['C', 'N', 'O', 'F', 'I', 'Cl', 'Br']
    atom_type = atom_type_list[node_colors[node]]
    node_labels.append(f"{atom_type}")
    rgba_color = colors[node_colors[node]]
    plotly_color = f"rgba({rgba_color[0] * 255}, {rgba_color[1] * 255}, {rgba_color[2] * 255}, {rgba_color[3]})"
    node_colors_plotly.append(plotly_color)

node_trace_2d = go.Scatter(
    x=node_x_2d, y=node_y_2d,
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        size=10,
        color=node_colors_plotly,
    ),
    text=node_labels,
    textposition="top center"
)

# 2D Visualization
fig_2d = go.Figure(data=[edge_trace_2d, node_trace_2d],
                   layout=go.Layout(
                       title="2D Visualization of Graph 0",
                       showlegend=False,
                       hovermode='closest',
                       margin=dict(b=0, l=0, r=0, t=40),
                       xaxis=dict(showgrid=False, zeroline=False),
                       yaxis=dict(showgrid=False, zeroline=False)
                   ))
fig_2d.show()

# Analyze graph properties
print(f"Number of nodes: {single_graph.num_nodes}")
print(f"Number of edges: {single_graph.num_edges}")
print(f"Is graph directed? {single_graph.is_directed()}")
print(f"Graph label: {single_graph.y}")


Number of nodes: 17
Number of edges: 38
Is graph directed? False
Graph label: tensor([1])


In [33]:
import torch
import plotly.graph_objs as go
import networkx as nx
from torch_geometric.utils import to_networkx
import numpy as np
import matplotlib.pyplot as plt

single_graph = dataset[4]
# Assuming you already have the graph data loaded as `single_graph`
# Convert the first graph in the dataset to NetworkX.
G = to_networkx(single_graph, to_undirected=True)

# Extract the node feature matrix
node_features = single_graph.x  # Shape: [num_nodes, num_features]

# Convert one-hot encoded node features to atom types (indices)
node_colors = torch.argmax(node_features, dim=1).numpy()

# Define atom labels and fixed colormap
atom_labels = ['C', 'N', 'O', 'F', 'I', 'Cl', 'Br']
fixed_colors = {
    'C': 'rgba(0, 255, 0, 1)',  # Green
    'N': 'rgba(255, 0, 0, 1)',  # Red
    'O': 'rgba(0, 0, 255, 1)',  # Blue
    'F': 'rgba(255, 255, 0, 1)',  # Yellow
    'I': 'rgba(128, 0, 128, 1)',  # Purple
    'Cl': 'rgba(255, 165, 0, 1)',  # Orange
    'Br': 'rgba(75, 0, 130, 1)'   # Indigo
}

# Generate 3D layout
pos_3d = nx.spring_layout(G, dim=3, seed=42)

# Extract node labels based on atom types
node_labels = []
node_colors_plotly = []
for node in G.nodes():
    atom_type = atom_labels[node_colors[node]]
    node_labels.append(atom_type)
    plotly_color = fixed_colors[atom_type]
    node_colors_plotly.append(plotly_color)

# Prepare edge trace for 3D visualization
edge_x_3d = []
edge_y_3d = []
edge_z_3d = []
for edge in G.edges():
    x0, y0, z0 = pos_3d[edge[0]]
    x1, y1, z1 = pos_3d[edge[1]]
    edge_x_3d.extend([x0, x1, None])
    edge_y_3d.extend([y0, y1, None])
    edge_z_3d.extend([z0, z1, None])

edge_trace_3d = go.Scatter3d(
    x=edge_x_3d, y=edge_y_3d, z=edge_z_3d,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Prepare node trace for 3D visualization
node_x_3d = []
node_y_3d = []
node_z_3d = []
for node in G.nodes():
    x, y, z = pos_3d[node]
    node_x_3d.append(x)
    node_y_3d.append(y)
    node_z_3d.append(z)

node_trace_3d = go.Scatter3d(
    x=node_x_3d, y=node_y_3d, z=node_z_3d,
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        size=10,
        color=node_colors_plotly,
    ),
    text=node_labels,
    textposition="top center"
)

# 3D Visualization
fig_3d = go.Figure(data=[edge_trace_3d, node_trace_3d],
                   layout=go.Layout(
                       title="3D Visualization of Graph 0",
                       showlegend=False,
                       margin=dict(b=0, l=0, r=0, t=40),
                       scene=dict(
                           xaxis=dict(showticklabels=False),
                           yaxis=dict(showticklabels=False),
                           zaxis=dict(showticklabels=False)
                       )
                   ))
fig_3d.show()

# Analyze graph properties
print(f"Number of nodes: {single_graph.num_nodes}")
print(f"Number of edges: {single_graph.num_edges}")
print(f"Is graph directed? {single_graph.is_directed()}")
print(f"Graph label: {single_graph.y}")


Number of nodes: 11
Number of edges: 22
Is graph directed? False
Graph label: tensor([0])


In [34]:
i = 0
for d in dataset:
  if d.x[:, 4].sum().item() > 0:
    print(i)
  i+=1

21


In [35]:
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 150
Number of test graphs: 38


### PARTE DEL CINESE


In [36]:
import numpy as np
import scipy.sparse as sp
import torch


# MUTAG dataset features: 188 graphs, totaling 3371 nodes, 7442 edges, represented as an undirected graph

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def load_split_MUTAG_data(path="/content/datas/", dataset="MUTAG_", split_train=0.7, split_val=0.15):
    """Load MUTAG data"""
    print('Loading {} dataset...'.format(dataset))

    # Load graph labels
    graph_labels = np.genfromtxt("{}{}graph_labels.txt".format(path, dataset),
                                 dtype=np.dtype(int))
    graph_labels = encode_onehot(graph_labels)  # (188, 2)
    graph_labels = torch.LongTensor(np.where(graph_labels)[1])  # (188, 1)

    # Graph node indices
    graph_idx = np.genfromtxt("{}{}graph_indicator.txt".format(path, dataset),
                              dtype=np.dtype(int))

    graph_idx = np.array(graph_idx, dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(graph_idx)}  # key, value indicates the starting node index of the graph key is value
    length = len(idx_map.keys())  # Total number of graphs
    num_nodes = [idx_map[n] - idx_map[n - 1] if n - 1 > 1 else idx_map[n] for n in range(1, length + 1)]  # A list of length 188 representing the number of nodes in each graph
    max_num_nodes = max(num_nodes)  # Maximum number of nodes in a single graph
    features_list = []
    adj_list = []
    prev = 0

    # Node labels
    nodeidx_features = np.genfromtxt("{}{}node_labels.txt".format(path, dataset), delimiter=",",
                                     dtype=np.dtype(int))
    node_features = np.zeros((nodeidx_features.shape[0], max(nodeidx_features) + 1))
    node_features[np.arange(nodeidx_features.shape[0]), nodeidx_features] = 1

    # Edge information
    edges_unordered = np.genfromtxt("{}{}A.txt".format(path, dataset), delimiter=",",
                                    dtype=np.int32)

    # Edge labels
    edges_label = np.genfromtxt("{}{}edge_labels.txt".format(path, dataset), delimiter=",",
                                dtype=np.int32)  # shape = (7442,)

    # Generate adjacency matrix A, which includes all edges in the dataset
    adj = sp.coo_matrix((edges_label, (edges_unordered[:, 0] - 1, edges_unordered[:, 1] - 1)))

    # Formula in the paper: A^ = (D~)^0.5 A~ (D~)^0.5
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    node_features = normalize(node_features)
    adj = normalize(adj + sp.eye(adj.shape[0]))  # Corresponds to the formula A~ = A + IN
    adj = adj.todense()

    for n in range(1, length + 1):
        # 'entry' is the feature matrix X for the n-th graph
        entry = np.zeros((max_num_nodes, max(nodeidx_features) + 1))
        entry[:idx_map[n] - prev] = node_features[prev:idx_map[n]]
        entry = torch.FloatTensor(entry)
        features_list.append(entry.tolist())

        # 'entry' is the adjacency matrix A for the n-th graph
        entry = np.zeros((max_num_nodes, max_num_nodes))
        entry[:idx_map[n] - prev, :idx_map[n] - prev] = adj[prev:idx_map[n], prev:idx_map[n]]
        entry = torch.FloatTensor(entry)
        adj_list.append(entry.tolist())

        prev = idx_map[n]  # 'prev' is the starting node index for the next graph

    num_total = max(graph_idx)
    num_train = int(split_train * num_total)
    num_val = int((split_train + split_val) * num_total)

    if (num_train == num_val or num_val == num_total):
        return

    features_list = torch.FloatTensor(features_list)
    adj_list = torch.FloatTensor(adj_list)

    idx_train = range(num_train)
    idx_val = range(num_train, num_val)
    idx_test = range(num_val, num_total)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    # Return values in order: adjacency matrices list of 188 graphs, feature matrices list of 188 graphs, labels of 188 graphs, starting node index of each graph, training set indices, validation set indices, test set indices
    return adj_list, features_list, graph_labels, idx_map, idx_train, idx_val, idx_test


### GNN MODEL

In [37]:
#@title Model to Explain (GCN)
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter


class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    paper: Semi-Supervised Classification with Graph Convolutional Networks
    """
    # The parameters of the model include weight and bias
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))  # Weight matrix for the GCN layer
        self.bias = Parameter(torch.FloatTensor(out_features))  # Bias term for the GCN layer
        self.reset_parameters()  # Initialize parameters

    # Weight initialization
    def reset_parameters(self):
        # Initialize weights and biases using uniform distribution based on the size of the output features
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    # Representation function, similar to __str__
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

    # Compute A~ X W(0), where A~ is the normalized adjacency matrix and X is the input feature matrix
    def forward(self, input, adj):
        # input.shape = [num_nodes, features] = X
        # adj.shape = [num_nodes, num_nodes] = A~
        # torch.mm(a, b) performs matrix multiplication of a and b, torch.mul(a, b) performs element-wise multiplication (a and b must have the same dimensions)
        support = torch.mm(input, self.weight)  # Matrix multiplication of input features with weights, shape = [max_node, out_features]
        output = torch.spmm(adj, support)  # Multiply normalized adjacency matrix with the support matrix, shape = [max_node, out_features]
        return output + self.bias  # Add bias term, shape = [max_node, out_features]


class GCN(nn.Module):
    # nfeat: number of input features, nclass: number of classes for classification, dropout: dropout rate
    def __init__(self, nfeat, nclass, dropout):
        """ As per paper """
        """ 3 layers of GCNs with output dimensions equal to 32, 48, 64 respectively and average all node features """
        """ Final classifier with 2 fully connected layers and hidden dimension set to 32 """
        """ Activation function - ReLu (Mutag) """
        super(GCN, self).__init__()

        self.dropout = dropout

        # Define three GCN layers with increasing output dimensions
        self.gc1 = GraphConvolution(nfeat, 32)  # First GCN layer (input features -> 32)
        self.gc2 = GraphConvolution(32, 48)  # Second GCN layer (32 -> 48)
        self.gc3 = GraphConvolution(48, 64)  # Third GCN layer (48 -> 64)

        # Define two fully connected (linear) layers for classification
        self.fc1 = nn.Linear(64, 32)  # First fully connected layer (64 -> 32)
        self.fc2 = nn.Linear(32, nclass)  # Second fully connected layer (32 -> nclass)

    def forward(self, x, adj):
        # x.shape = [max_node, features]
        # adj.shape = [max_node, max_node]

        # First GCN layer with ReLU activation and dropout
        x = F.relu(self.gc1(x, adj))  # x.shape = [num_nodes, 32]
        x = F.dropout(x, self.dropout, training=self.training)  # Apply dropout

        # Second GCN layer with ReLU activation and dropout
        x = F.relu(self.gc2(x, adj))  # x.shape = [num_nodes, 48]
        x = F.dropout(x, self.dropout, training=self.training)  # Apply dropout

        # Third GCN layer with ReLU activation
        x = F.relu(self.gc3(x, adj))  # x.shape = [num_nodes, 64]

        # Aggregate all node features by taking the mean of all nodes
        y = torch.mean(x, 0)  # y.shape = [64], aggregate using mean over all nodes

        # Apply fully connected layers for final classification
        y = F.relu(self.fc1(y))  # First fully connected layer with ReLU activation, y.shape = [32]
        y = F.dropout(y, self.dropout, training=self.training)  # Apply dropout
        y = F.softmax(self.fc2(y), dim=0)  # Second fully connected layer with softmax activation, y.shape = [nclass]

        return y


if __name__ == '__main__':
    # Random input features of 29 nodes, each with 7 features
    input = torch.rand(29, 7)
    # Random adjacency matrix of size 29x29
    adj = torch.rand(29, 29)

    # Initialize the GCN model with 7 input features, 2 output classes, and a dropout rate of 0.1
    model = GCN(nfeat=7,  # nfeat = 7
                nclass=2,  # nclass = 2
                dropout=0.1)

    # Forward pass through the model
    output = model(input, adj)
    print(output.size())  # Output size should be [nclass], which is [2] in this case


torch.Size([2])


In [38]:
#@title Dataset reshaping for training
import torch
import numpy as np
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_dense_adj

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def load_dataset_and_split(train_percent=0.8):
    # Prepare the output tensors
    num_graphs = len(dataset)
    max_nodes = 29  # Given that 29 is the maximum number of nodes in a single graph
    num_features = 7

    # Initialize the lists to hold the reshaped data
    adj_list = torch.zeros((num_graphs, max_nodes, max_nodes))  # [188, 29, 29]
    features_list = torch.zeros((num_graphs, max_nodes, num_features))  # [188, 29, 7]
    graph_labels = torch.zeros(num_graphs, dtype=torch.long)  # [188]

    # Iterate over each graph in the dataset
    for i, data in enumerate(dataset):
        # Number of nodes in the current graph
        num_nodes = data.num_nodes

        # Extract the node feature matrix x and store it in features_list
        features = data.x  # Shape: [num_nodes, num_features]
        features_list[i, :num_nodes, :] = features  # Padding remaining nodes with zeros

        # Create adjacency matrix from edge index and add self-loops
        adj = to_dense_adj(data.edge_index, max_num_nodes=max_nodes)[0]  # Convert edge index to dense adjacency matrix
        adj = adj + torch.eye(max_nodes)  # Add self-loops by adding an identity matrix
        adj_list[i] = adj

        # Store the graph label
        graph_labels[i] = data.y

    # Create train-val-test split indices
    num_train = int(train_percent * num_graphs)  # 80% of the data for training
    num_val = num_graphs - num_train

    # Generate shuffled indices
    indices = torch.randperm(num_graphs)

    # Assign indices to each split
    idx_train = indices[:num_train]
    idx_val = indices[num_train:]

    # Print the shapes to confirm
    print('adj_list:', adj_list.shape)  # Should be [188, 29, 29]
    print('features_list:', features_list.shape)  # Should be [188, 29, 7]
    print('graph_labels:', graph_labels.shape)  # Should be [188]
    print('idx_train:', idx_train.shape)  # Should be [80% of 188]
    print('idx_val:', idx_val.shape)  # Should be [20% of 188]

    return adj_list, features_list, graph_labels, idx_train, idx_val

In [39]:
#@title Training the GCN
import time

import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F

model_path = 'model/gcn_first.pth'

epochs = 1000
lr = 0.001
dropout = 0.2
weight_decay = 5e-4

class EarlyStopping():
    def __init__(self, patience=10, min_loss=0.5, hit_min_before_stopping=False):
        self.patience = patience
        self.counter = 0
        self.hit_min_before_stopping = hit_min_before_stopping
        if hit_min_before_stopping:
            self.min_loss = min_loss
        self.best_loss = None
        self.early_stop = False

    def __call__(self, loss):
        if self.best_loss is None:
            self.best_loss = loss
        elif loss > self.best_loss:
            self.counter += 1
            if self.counter > self.patience:
                if self.hit_min_before_stopping == True and loss > self.min_loss:
                    print("Cannot hit min loss, will continue")
                    self.counter -= self.patience
                else:
                    self.early_stop = True
        else:
            self.best_loss = loss
            counter = 0


if __name__ == '__main__':
    # adj_list: [188, 29, 29]
    # features_list: [188, 29, 7]
    # graph_labels: [188]
    adj_list, features_list, graph_labels, idx_train, idx_val = load_dataset_and_split()

    model = GCN(nfeat=features_list[0].shape[1], # nfeat = 7
                nclass=graph_labels.max().item() + 1, # nclass = 2
                dropout=dropout)
    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)

    model.cuda()
    features_list = features_list.cuda()
    adj_list = adj_list.cuda()
    graph_labels = graph_labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()

    early_stopping = EarlyStopping(patience=10, hit_min_before_stopping=True)
    t_total = time.time()

    for epoch in range(epochs):
        t = time.time()
        model.train()
        optimizer.zero_grad()

        # # Forward pass through entire training set
        outputs = []
        outputs = [model(features_list[i], adj_list[i]).unsqueeze(0) for i in idx_train]
        output = torch.cat(outputs, dim=0)

        loss_train = F.cross_entropy(output, graph_labels[idx_train])
        acc_train = accuracy(output, graph_labels[idx_train])
        loss_train.backward()
        optimizer.step()

        model.eval()
        # # Forward pass through entire validation set
        outputs = []
        outputs = [model(features_list[i], adj_list[i]).unsqueeze(0) for i in idx_val]
        output = torch.cat(outputs, dim=0)
        loss_val = F.cross_entropy(output, graph_labels[idx_val])
        acc_val = accuracy(output, graph_labels[idx_val])

        print('Epoch: {:04d}'.format(epoch + 1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

        early_stopping(loss_val)
        if early_stopping.early_stop == True:
            break

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

    torch.save(model.state_dict(), model_path)

adj_list: torch.Size([188, 29, 29])
features_list: torch.Size([188, 29, 7])
graph_labels: torch.Size([188])
idx_train: torch.Size([150])
idx_val: torch.Size([38])
Epoch: 0001 loss_train: 0.6800 acc_train: 0.6400 loss_val: 0.6582 acc_val: 0.7632 time: 1.6837s
Epoch: 0002 loss_train: 0.6745 acc_train: 0.6400 loss_val: 0.6516 acc_val: 0.7632 time: 0.6131s
Epoch: 0003 loss_train: 0.6697 acc_train: 0.6400 loss_val: 0.6462 acc_val: 0.7632 time: 0.4343s
Epoch: 0004 loss_train: 0.6652 acc_train: 0.6400 loss_val: 0.6412 acc_val: 0.7632 time: 0.3978s
Epoch: 0005 loss_train: 0.6627 acc_train: 0.6400 loss_val: 0.6357 acc_val: 0.7632 time: 0.5084s
Epoch: 0006 loss_train: 0.6601 acc_train: 0.6400 loss_val: 0.6297 acc_val: 0.7632 time: 0.3923s
Epoch: 0007 loss_train: 0.6550 acc_train: 0.6400 loss_val: 0.6234 acc_val: 0.7632 time: 0.4327s
Epoch: 0008 loss_train: 0.6495 acc_train: 0.6400 loss_val: 0.6167 acc_val: 0.7632 time: 0.6473s
Epoch: 0009 loss_train: 0.6476 acc_train: 0.6400 loss_val: 0.6096 acc

RuntimeError: Parent directory model does not exist.

### GRAPH GENERATOR

In [41]:
import random
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F


rollout = 10
MAX_NUM_NODES = 29 # for mutag

class Generator(nn.Module):
    def __init__(self, model_path: str, C: list, node_feature_dim: int ,num_class = 2, c=0, hyp1=1, hyp2=2, start=None, nfeat=7, dropout=0.1):
        """
        :param C: Candidate set of nodes (list)
        :param start: Starting node (defaults to randomized node)
        """
        super(Generator, self).__init__()
        self.nfeat = nfeat
        self.dropout = dropout
        self.c = c # c is the specific class of graph to be generated

        self.fc = nn.Linear(nfeat, 8)
        self.gc1 = GraphConvolution(8, 16)
        self.gc2 = GraphConvolution(16, 24)
        self.gc3 = GraphConvolution(24, 32)

        # MLP1
        # 2 FC layers with hidden dimension 16
        self.mlp1 = nn.Sequential(nn.Linear(32, 16), nn.Linear(16, 1))

        # MLP2
        # 2 FC layers with hidden dimension 24
        self.mlp2 = nn.Sequential(nn.Linear(64, 24), nn.Linear(24, 1))

        # Hyperparameters
        self.hyp1 = hyp1
        self.hyp2 = hyp2
        self.candidate_set = C

        # Default starting node (if any)
        if start != None:
            self.start = start
            self.random_start = False
        else:
            self.start = random.choice(np.arange(0, len(self.candidate_set)))
            self.random_start = True

        # Load GCN for calculating reward
        self.model = GCN(nfeat=node_feature_dim,
                         nclass=num_class,
                         dropout=dropout)

        self.model.load_state_dict(torch.load(model_path))
        for param in self.model.parameters():
            param.requires_grad = False

        self.reset_graph()

    def reset_graph(self):
        """
        Reset g.G to default graph with only the start node, generate a graph containing only one node
        """
        if self.random_start == True:
            self.start = random.choice(np.arange(0, len(self.candidate_set)))

        # In the initial graph, all nodes except the first one are masked. Since the adjacency matrix's edge length is MAX_NUM_NODES + len(self.candidate_set), not only the candidate nodes are masked, but also all virtual nodes in the graph
        mask_start = torch.BoolTensor(
            [False if i == 0 else True for i in range(MAX_NUM_NODES + len(self.candidate_set))])

        adj = torch.zeros((MAX_NUM_NODES + len(self.candidate_set), MAX_NUM_NODES + len(self.candidate_set)),
                          dtype=torch.float32)   # Here, the shape of adj is [MAX_NUM_NODES + len(self.candidate_set), MAX_NUM_NODES + len(self.candidate_set)], which may contain empty nodes

        feat = torch.zeros((MAX_NUM_NODES + len(self.candidate_set), len(self.candidate_set)), dtype=torch.float32)
        # the first node in the feature matrix is the starting node -> one hot encoding to make its features
        feat[0, self.start] = 1
        # this fills the bottom part of node feature matrix "x" with the candidate nodes features (just the one hot encoding for each type of node)
        feat[np.arange(-len(self.candidate_set), 0), np.arange(0, len(self.candidate_set))] = 1
        # the remaining MAX_NUM_NODES - len(candidate_set) - 1(start node) features are empty, all zeros, since we start with a single node and the candidate set

        degrees = torch.zeros(MAX_NUM_NODES)

        self.G = {'adj': adj, 'feat': feat, 'degrees': degrees, 'num_nodes': 1, 'mask_start': mask_start}

    ## Calculate Gt -> Gt+1
    def forward(self, G_in):
        ## G_in is Gt
        G = copy.deepcopy(G_in)

        x = G['feat'].detach().clone() # Feature matrix of Gt
        adj = G['adj'].detach().clone() # Adjacency matrix of Gt

        ## Corresponds to X = GCNs(Gt​,C)
        x = F.relu6(self.fc(x))
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu6(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu6(self.gc2(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu6(self.gc3(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)

        ## pt,start=Softmax(MLPs(X))
        p_start = self.mlp1(x)
        p_start = p_start.masked_fill(G['mask_start'].unsqueeze(1), float('-inf'))
        p_start = F.softmax(p_start, dim=0)
        a_start_idx = torch.argmax(p_start)

        ## pt,end=Softmax(MLPs([X,x^start​))
        # broadcast
        x1, x2 = torch.broadcast_tensors(x, x[a_start_idx])
        x = torch.cat((x1, x2), 1)  # cat increases dim from 32 to 64

        # Calculate mask_end, except for the candidate set and nodes in Gt that have not been selected as starting nodes, all others are masked
        mask_end = torch.BoolTensor([True for i in range(MAX_NUM_NODES + len(self.candidate_set))])
        mask_end[MAX_NUM_NODES:] = False
        mask_end[:G['num_nodes']] = False
        mask_end[a_start_idx] = True

        p_end = self.mlp2(x)
        p_end = p_end.masked_fill(mask_end.unsqueeze(1), float('-inf'))
        p_end = F.softmax(p_end, dim=0)
        a_end_idx = torch.argmax(p_end)

        # Return new G
        # If a_end_idx is not masked, the node exists in the graph, no new node added
        if G['mask_start'][a_end_idx] == False:
            G['adj'][a_end_idx][a_start_idx] += 1
            G['adj'][a_start_idx][a_end_idx] += 1

            # Update degrees
            G['degrees'][a_start_idx] += 1
            G['degrees'][a_end_idx] += 1 # ???? original line: G['degrees'][G['num_nodes']] += 1
        else: # if end-node was chosen from candidate set:
            # Add node
            G['feat'][G['num_nodes']] = G['feat'][a_end_idx]
            # Add edge
            G['adj'][G['num_nodes']][a_start_idx] += 1
            G['adj'][a_start_idx][G['num_nodes']] += 1
            # Update degrees
            G['degrees'][a_start_idx] += 1
            G['degrees'][G['num_nodes']] += 1

            # Update start mask
            G_mask_start_copy = G['mask_start'].detach().clone()
            G_mask_start_copy[G['num_nodes']] = False
            G['mask_start'] = G_mask_start_copy

            G['num_nodes'] += 1

        return p_start, a_start_idx, p_end, a_end_idx, G


    ### Reward function
    def calculate_reward(self, G_t_1):
        """
        Rtr     Calculated from graph rules to encourage generated graphs to be valid
                1. Only one edge to be added between any two nodes
                2. Generated graph cannot contain more nodes than predefined maximum node number
                3. (For chemical) Degree cannot exceed valency
                If generated graph violates graph rule, Rtr = -1

        Rtf     Feedback from trained model
        """

        rtr = self.check_graph_rules(G_t_1)

        rtf = self.calculate_reward_feedback(G_t_1)
        rtf_sum = 0
        for m in range(rollout):
            p_start, a_start, p_end, a_end, G_t_1 = self.forward(G_t_1)
            rtf_sum += self.calculate_reward_feedback(G_t_1)
        rtf = rtf + rtf_sum * self.hyp1 / rollout

        return rtf + self.hyp2 * rtr

    def calculate_reward_feedback(self, G_t_1):
        """
        p(f(G_t_1) = c) - 1/l
        where l denotes number of possible classes for f
        """
        f = self.model(G_t_1['feat'], G_t_1['adj'])
        return f[self.c] - 1 / len(f)


    ## Graph rules
    def check_graph_rules(self, G_t_1):
        """
        For mutag, node degrees cannot exceed valency
        """
        idx = 0

        for d in G_t_1['degrees']:
            if d != 0:
                node_id = torch.argmax(G_t_1['feat'][idx])  # Eg. [0, 1, 0, 0] -> 1
                node = self.candidate_set[node_id]  # Eg ['C.4', 'F.2', 'Br.7'][1] = 'F.2'
                max_valency = int(node.split('.')[1])  # Eg. C.4 -> ['C', '4'] -> 4

                # If any node degree exceeds its valency, return -1
                if max_valency < d:
                    return -1

        return 0


    ## Calculate loss
    def calculate_loss(self, Rt, p_start, a_start, p_end, a_end, G_t_1):
        """
        Calculated from cross entropy loss (Lce) and reward function (Rt)
        where loss = -Rt*(Lce_start + Lce_end)
        """

        Lce_start = F.cross_entropy(torch.reshape(p_start, (1, 35)), a_start.unsqueeze(0))
        Lce_end = F.cross_entropy(torch.reshape(p_end, (1, 35)), a_end.unsqueeze(0))

        return -Rt * (Lce_start + Lce_end)


### CHECK

In [ ]:
import numpy as np
import scipy.sparse as sp

import torch

src_path = '/content/datas/MUTAG_{}'
split_train=0.7
split_val=0.15

if __name__ == '__main__':
    nodeidx_features = np.genfromtxt(src_path.format('node_labels.txt'), delimiter=",",
                                     dtype=np.dtype(int))
    node_features = np.zeros((nodeidx_features.shape[0], max(nodeidx_features) + 1))
    node_features[np.arange(nodeidx_features.shape[0]), nodeidx_features] = 1


    graph_labels = np.genfromtxt(src_path.format('graph_labels.txt'), dtype=np.dtype(int))
    graph_labels = encode_onehot(graph_labels)
    graph_labels = torch.LongTensor(np.where(graph_labels)[1])


    graph_idx = np.genfromtxt(src_path.format('graph_indicator.txt'),dtype=np.dtype(int))
    graph_idx = np.array(graph_idx, dtype=np.int32)


    edges_unordered = np.genfromtxt(src_path.format('A.txt'), delimiter=",",
                                    dtype=np.int32)  # (7442,2)

    edges_label = np.genfromtxt(src_path.format('edge_labels.txt'), delimiter=",",
                                dtype=np.int32)

    # 邻接矩阵
    adj = sp.coo_matrix((edges_label, (edges_unordered[:, 0] - 1, edges_unordered[:, 1] - 1)))
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj) # (3371, 3371)

    idx_map = {j: i for i, j in enumerate(graph_idx)} # key, value表示第key个图的起始结点索引号为value
    length = len(idx_map.keys())  # 总共有多少个图 , 188
    num_nodes = [idx_map[n] - idx_map[n - 1] if n - 1 > 1 else idx_map[n] for n in range(1, length + 1)]  # 一个长度188的list，表示没个图有多少个结点
    max_num_nodes = max(num_nodes) # 最大的一个图有多少个结点 实际29
    features_list = []
    adj_list = []
    prev = 0

    node_features = normalize(node_features) # (3371, 7)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    adj = adj.todense()

    for n in range(1, length + 1):
        # entry为图的特征矩阵X
        entry = np.zeros((max_num_nodes, max(nodeidx_features) + 1))
        entry[:idx_map[n] - prev] = node_features[prev:idx_map[n]]
        entry = torch.FloatTensor(entry)
        features_list.append(entry)

        # entry为图的邻接矩阵A
        entry = np.zeros((max_num_nodes, max_num_nodes))
        entry[:idx_map[n] - prev, :idx_map[n] - prev] = adj[prev:idx_map[n], prev:idx_map[n]]
        entry = torch.FloatTensor(entry)
        adj_list.append(entry)

        prev = idx_map[n] # prev为下个图起始结点的索引号

    num_total = max(graph_idx)
    num_train = int(split_train * num_total)
    num_val = int((split_train + split_val) * num_total)

    idx_train = range(num_train)
    idx_val = range(num_train, num_val)
    idx_test = range(num_val, num_total)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)


    print(graph_labels[idx_train])
    print(graph_labels[idx_val])

tensor([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
        0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1])
tensor([1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 0])


### TRAINING GENERATOR

In [42]:
import copy

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import torch
import torch.optim as optim

lr = 0.01
b1 = 0.9
b2 = 0.99
hyp1 = 1
hyp2 = 2
max_gen_step = 10  # T = 10

candidate_set = ['C.4', 'N.5', 'O.2', 'F.1', 'I.7', 'Cl.7', 'Br.5']  # C.4表明碳原子的度不超过4
model_path = 'model/gcn_first.pth'

## 训练generator
def train_generator(c=0, max_nodes=5):
    g.c = c
    for i in range(max_gen_step):
        optimizer.zero_grad()
        G = copy.deepcopy(g.G)
        p_start, a_start, p_end, a_end, G = g.forward(G)

        Rt = g.calculate_reward(G)
        loss = g.calculate_loss(Rt, p_start, a_start, p_end, a_end, G)
        loss.backward()
        optimizer.step()

        if G['num_nodes'] > max_nodes:
            g.reset_graph()
        elif Rt > 0:
            g.G = G


## 生成图
def generate_graph(c=0, max_nodes=5):
    g.c = c
    g.reset_graph()

    for i in range(max_gen_step):
        G = copy.deepcopy(g.G)
        p_start, a_start, p_end, a_end, G = g.forward(G)
        Rt = g.calculate_reward(G)

        if G['num_nodes'] > max_nodes:
            return g.G
        elif Rt > 0:
            g.G = G

    return g.G

## 画图
def display_graph(G):
    G_nx = nx.from_numpy_array(np.asmatrix(G['adj'][:G['num_nodes'], :G['num_nodes']].numpy()))
    # nx.draw_networkx(G_nx)

    layout=nx.spring_layout(G_nx)
    nx.draw(G_nx, layout)

    coloring=torch.argmax(G['feat'],1)
    colors=['b','g','r','c','m','y','k']

    for i in range(7):
        nx.draw_networkx_nodes(G_nx,pos=layout,nodelist=[x for x in G_nx.nodes() if coloring[x]==i],node_color=colors[i])
        nx.draw_networkx_labels(G_nx,pos=layout,labels={x:candidate_set[i].split('.')[0] for x in G_nx.nodes() if coloring[x]==i})
    nx.draw_networkx_edges(G_nx,pos=layout,width=list(nx.get_edge_attributes(G_nx,'weight').values()))
    nx.draw_networkx_edge_labels(G_nx,pos=layout,edge_labels=nx.get_edge_attributes(G_nx, "weight"))

    plt.show()

if __name__ == '__main__':
    g = Generator(model_path = model_path, C = candidate_set, node_feature_dim=7 ,c=0, start=0)
    optimizer = optim.Adam(g.parameters(), lr=lr, betas=(b1, b2))

    for i in range(1, 10):
        ## 生成最多分别包括i个结点的图结构
        g.reset_graph()
        train_generator(c=1, max_nodes=i)
        to_display = generate_graph(c=1, max_nodes=i)
        display_graph(to_display)
        print(g.model(to_display['feat'], to_display['adj']))

<ipython-input-41-89d7f46f373e>:55: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



RuntimeError: shape '[1, 35]' is invalid for input of size 36

### TEST

In [ ]:
from Load_dataset import load_split_MUTAG_data, accuracy
from Model import GCN
import time

import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F

model_path = 'model/gcn_first.pth'

if __name__ == '__main__':
    adj_list, features_list, graph_labels, idx_map, idx_train, idx_val, idx_test = load_split_MUTAG_data()
    model = GCN(nfeat=features_list[0].shape[1],  # nfeat = 7
                nclass=graph_labels.max().item() + 1,  # nclass = 2
                dropout=0.1)

    model.eval()
    outputs = []
    for i in idx_test:
        output = model(features_list[i], adj_list[i])
        output = output.unsqueeze(0)
        outputs.append(output)
    output = torch.cat(outputs, dim=0)

    loss_test = F.cross_entropy(output, graph_labels[idx_test])
    acc_test = accuracy(output, graph_labels[idx_test])
    print(loss_test)
    print(acc_test)

ModuleNotFoundError: No module named 'Load_dataset'